In [1]:
import psycopg2
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Connect to Database

In [3]:
#Connect to PgAdmin
engine = psycopg2.connect(
    database="project-data",
    user = "postgres",
    password = "Post23!Fnord",
    host = "happinessproject.cqkpnj5dcmou.us-east-1.rds.amazonaws.com",
    port = '5432'
    )

cursor = engine.cursor()


In [4]:
#Pull combined data table
query = "SELECT * from combined_data"

df = pd.read_sql(query, con=engine)

In [5]:
df.head()

,country_name,region,ladder_score,stand_err,up_whisker,low_whisker,log_gdp_per_cap,social_support,healty_life_exp,freedom,...,ladder_2011,ladder_2012,ladder_2013,ladder_2014,ladder_2015,ladder_2016,ladder_2017,ladder_2018,ladder_2019,ladder_2020
0,Afghanistan,South Asia,2.523,0.038,2.596,2.449,7.695,0.463,52.493,0.382,...,3.832,3.783,3.572,3.131,3.983,4.220,2.662,2.694,2.375,NaN
1,Albania,Central and Eastern Europe,5.117,0.059,5.234,5.001,9.520,0.697,68.999,0.785,...,5.867,5.510,4.551,4.814,4.607,4.511,4.640,5.004,4.995,5.365
2,Algeria,Middle East and North Africa,4.887,0.053,4.991,4.783,9.342,0.802,66.005,0.480,...,5.317,5.605,NaN,6.355,NaN,5.341,5.249,5.043,4.745,NaN
3,Argentina,Latin America and Caribbean,5.929,0.056,6.040,5.819,9.962,0.898,69.000,0.828,...,6.776,6.468,6.582,6.671,6.697,6.427,6.039,5.793,6.086,5.901
4,Armenia,Commonwealth of Independent States,5.283,0.058,5.397,5.168,9.487,0.799,67.055,0.825,...,4.260,4.320,4.277,4.453,4.348,4.325,4.288,5.062,5.488,NaN


# Clean master table for Machine Learning

In [6]:
#column names
list(df)

['country_name',
 'region',
 'ladder_score',
 'stand_err',
 'up_whisker',
 'low_whisker',
 'log_gdp_per_cap',
 'social_support',
 'healty_life_exp',
 'freedom',
 'generosity',
 'percept_corrupt',
 'ladder_score_distopia',
 'expby_log_gdp_per_cap',
 'expby_social_support',
 'expby_healty_life_exp',
 'expby_freedom',
 'expby_generosity',
 'expby_percept_corrupt',
 'distopia_plus_resid',
 'ac_both_sexes',
 'ac_male',
 'ac_female',
 'population_2021',
 'covid_total_cases',
 'covid_new_cases',
 'covid_total_deaths',
 'covid_new_deaths',
 'covid_total_recovered',
 'covid_new_recovered',
 'covid_active_cases',
 'covid_serious_critical',
 'covid_cases_per_mil',
 'covid_deaths_per_mil',
 'covid_total_tests',
 'covid_tests_per_mil',
 'covid_who_region',
 'population_2020',
 'land_area_skm',
 'density_skm',
 'meat_consumption',
 'median_age',
 'screen_time_avg',
 'suicide_rate',
 'ladder_2010',
 'ladder_2011',
 'ladder_2012',
 'ladder_2013',
 'ladder_2014',
 'ladder_2015',
 'ladder_2016',
 'ladde

In [7]:
#Drop columns not needed for analysis
df = df.drop(columns=["population_2020", "ladder_2010", "ladder_2012","ladder_2013", "ladder_2014", "ladder_2015", "ladder_2016",
                      "ladder_2017", "ladder_2018", "ladder_2019", "ladder_2020", "country_name", "up_whisker", "low_whisker",
                      "expby_log_gdp_per_cap", "expby_social_support", "expby_healty_life_exp", "expby_freedom", "expby_generosity",
                      "expby_percept_corrupt", "stand_err", "distopia_plus_resid", "ac_both_sexes", "region"])

In [8]:
# view total null values for columns
df.isna().sum()

ladder_score                0
log_gdp_per_cap             0
social_support              0
healty_life_exp             0
freedom                     0
generosity                  0
percept_corrupt             0
ladder_score_distopia       0
ac_male                     8
ac_female                   8
population_2021             7
covid_total_cases           7
covid_new_cases           145
covid_total_deaths         10
covid_new_deaths          146
covid_total_recovered      11
covid_new_recovered       146
covid_active_cases         11
covid_serious_critical     52
covid_cases_per_mil         7
covid_deaths_per_mil       10
covid_total_tests          17
covid_tests_per_mil        17
covid_who_region            7
land_area_skm               8
density_skm                 8
meat_consumption            9
median_age                  5
screen_time_avg           106
suicide_rate                6
ladder_2011                20
dtype: int64

In [9]:
#compare NA values to dataset row count
len(df)

149

In [10]:
#drop columns with high NA count (Na count > 75)
df = df.drop(columns="covid_new_cases")
df = df.drop(columns="covid_new_deaths")
df = df.drop(columns="covid_new_recovered")
df = df.drop(columns="screen_time_avg")

In [11]:
# review total null values for columns
df.isna().sum()

ladder_score               0
log_gdp_per_cap            0
social_support             0
healty_life_exp            0
freedom                    0
generosity                 0
percept_corrupt            0
ladder_score_distopia      0
ac_male                    8
ac_female                  8
population_2021            7
covid_total_cases          7
covid_total_deaths        10
covid_total_recovered     11
covid_active_cases        11
covid_serious_critical    52
covid_cases_per_mil        7
covid_deaths_per_mil      10
covid_total_tests         17
covid_tests_per_mil       17
covid_who_region           7
land_area_skm              8
density_skm                8
meat_consumption           9
median_age                 5
suicide_rate               6
ladder_2011               20
dtype: int64

In [12]:
#drop all rows containing NA values
df = df.dropna()

In [13]:
# view total null values for columns
df.isna().sum()

ladder_score              0
log_gdp_per_cap           0
social_support            0
healty_life_exp           0
freedom                   0
generosity                0
percept_corrupt           0
ladder_score_distopia     0
ac_male                   0
ac_female                 0
population_2021           0
covid_total_cases         0
covid_total_deaths        0
covid_total_recovered     0
covid_active_cases        0
covid_serious_critical    0
covid_cases_per_mil       0
covid_deaths_per_mil      0
covid_total_tests         0
covid_tests_per_mil       0
covid_who_region          0
land_area_skm             0
density_skm               0
meat_consumption          0
median_age                0
suicide_rate              0
ladder_2011               0
dtype: int64

In [15]:
#length of final dataset for analysis 
len(df)

80

# Random Forest: Find the 10 variables that impact happiness scores the most 

In [16]:
# Create columns and target variables
columns = [
    "country_name", "region", "ladder_score", "stand_err",
    "up_whisker", "low_whisker", "log_gdp_per_cap", "social_support",
    "expby_social_support", "freedom", "generosity", "percept_corrupt",
    "ladder_score_distopia", "expby_log_gdp_per_cap", "expby_social_support", "expby_healty_life_exp",
    "expby_freedom", "expby_generosity", "expby_percept_corrupt", "distopia_plus_resid", "ac_both_sexes", "ac_male",
    "ac_female", "population_2021", "covid_total_cases", "covid_new_cases", "covid_total_deaths", "covid_total_recovered",
    "covid_new_recovered", "covid_active_cases", "covid_serious_critical", "covid_cases_per_mil", "covid_deaths_per_mil",
    "covid_total_tests", "covid_tests_per_mil", "covid_who_region", "population_2020", "land_area_skm", "land_area_skm", 
    "density_skm", "meat_consumption", "median_age", "screen_time_avg", "suicide_rate", "ladder_2010", "ladder_2012", 
    "ladder_2013", "ladder_2014", "ladder_2015", "ladder_2016", "ladder_2017", "ladder_2018", "ladder_2019", "ladder_2020"
]

target = ["ladder_score"]

In [17]:
#Convert variable type for target to non-continuous dtype
df = df.astype({'ladder_score':'int'})

df.head()

,ladder_score,log_gdp_per_cap,social_support,healty_life_exp,freedom,generosity,percept_corrupt,ladder_score_distopia,ac_male,ac_female,...,covid_deaths_per_mil,covid_total_tests,covid_tests_per_mil,covid_who_region,land_area_skm,density_skm,meat_consumption,median_age,suicide_rate,ladder_2011
0,2,7.695,0.463,52.493,0.382,-0.102,0.924,2.43,0.022,0.003,...,33.0,90396.0,2317.0,EasternMediterranean,652230.0,56.94,17.3,27.4,6.4,3.832
3,5,9.962,0.898,69.000,0.828,-0.182,0.834,2.43,14.660,4.640,...,94.0,794544.0,17564.0,Americas,2780400.0,16.26,79.7,31.7,9.1,6.776
5,7,10.796,0.940,73.900,0.914,0.159,0.442,2.43,15.720,5.110,...,10.0,4631419.0,181419.0,WesternPacific,7741220.0,3.25,108.2,38.7,11.7,7.406
6,7,10.906,0.934,73.300,0.908,0.042,0.481,2.43,18.420,5.760,...,80.0,937275.0,104008.0,Europe,83871.0,107.13,94.1,44.0,11.4,7.471
7,5,9.569,0.836,65.656,0.814,-0.223,0.506,2.43,1.670,0.320,...,47.0,766179.0,75499.0,Europe,86600.0,120.26,15.9,32.3,2.6,4.680


In [18]:
# Create our features
X = pd.get_dummies(df.drop(columns="ladder_score"))


# Create our target
y = df["ladder_score"]

In [19]:
X.describe()

,log_gdp_per_cap,social_support,healty_life_exp,freedom,generosity,percept_corrupt,ladder_score_distopia,ac_male,ac_female,population_2021,...,meat_consumption,median_age,suicide_rate,ladder_2011,covid_who_region_Africa,covid_who_region_Americas,covid_who_region_EasternMediterranean,covid_who_region_Europe,covid_who_region_South-EastAsia,covid_who_region_WesternPacific
count,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,8.000000e+01,80.000000,80.000000,8.000000e+01,...,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000
mean,9.662212,0.824138,66.246112,0.791963,-0.053387,0.760250,2.430000e+00,9.659525,2.721687,5.475099e+07,...,50.610000,31.685000,9.600000,5.586787,0.162500,0.237500,0.150000,0.350000,0.037500,0.062500
std,0.987616,0.113083,5.476313,0.114115,0.131526,0.150201,4.022020e-15,6.605509,2.053321,1.598690e+08,...,33.880866,8.454885,5.362575,1.118845,0.371236,0.428236,0.359324,0.479979,0.191182,0.243589
min,6.958000,0.463000,52.493000,0.382000,-0.288000,0.179000,2.430000e+00,0.000000,0.000000,6.269520e+05,...,5.100000,16.500000,2.200000,2.936000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.068250,0.782000,64.276000,0.739250,-0.147500,0.720000,2.430000e+00,3.570000,0.785000,6.912696e+06,...,22.650000,26.000000,5.425000,4.814250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9.737500,0.850500,67.051000,0.818000,-0.081000,0.801000,2.430000e+00,9.710000,2.430000,1.837278e+07,...,44.900000,30.400000,9.100000,5.485500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,10.391750,0.903000,69.664500,0.876250,0.033500,0.857000,2.430000e+00,15.580000,4.610000,4.408861e+07,...,72.550000,39.825000,12.900000,6.560500,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,11.647000,0.954000,75.100000,0.970000,0.311000,0.939000,2.430000e+00,20.610000,6.340000,1.381345e+09,...,145.900000,47.300000,26.500000,7.788000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
# Check the balance of our target values
y.value_counts()

5    26
6    22
4    19
7     9
3     3
2     1
Name: ladder_score, dtype: int64

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({6: 19, 5: 20, 4: 12, 7: 5, 3: 3, 2: 1})

In [22]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

# Creating a StandardScaler instance.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [23]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, predictions)
acc_score

0.5535714285714286

In [24]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[5, 1, 1, 0],
       [0, 2, 4, 0],
       [0, 1, 2, 0],
       [0, 0, 2, 2]], dtype=int64)

In [25]:
# Print the imbalanced classification report

from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          4       1.00      0.71      1.00      0.83      0.85      0.69         7
          5       0.50      0.33      0.86      0.40      0.53      0.27         6
          6       0.22      0.67      0.59      0.33      0.63      0.40         3
          7       1.00      0.50      1.00      0.67      0.71      0.48         4

avg / total       0.73      0.55      0.90      0.59      0.69      0.48        20



In [26]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.07691853845821622, 'percept_corrupt'),
 (0.07220577605212508, 'freedom'),
 (0.06873687984201149, 'covid_tests_per_mil'),
 (0.06686656782792265, 'log_gdp_per_cap'),
 (0.06654099021747768, 'ladder_2011'),
 (0.061810495791871145, 'healty_life_exp'),
 (0.0530904855309898, 'meat_consumption'),
 (0.04475860722872547, 'covid_total_tests'),
 (0.040888530694446645, 'social_support'),
 (0.038910586384426404, 'median_age'),
 (0.036848090560048154, 'land_area_skm'),
 (0.03422793186989428, 'covid_total_recovered'),
 (0.03397777990525528, 'covid_total_deaths'),
 (0.03370898028424446, 'covid_total_cases'),
 (0.03292320714445854, 'generosity'),
 (0.0309845340038858, 'covid_cases_per_mil'),
 (0.02553578817274522, 'suicide_rate'),
 (0.02532314582289678, 'covid_deaths_per_mil'),
 (0.02482010864296106, 'covid_active_cases'),
 (0.024819965124762703, 'ac_female'),
 (0.024046532812116127, 'density_skm'),
 (0.024040389226937187, 'ac_male'),
 (0.021783229621233132, 'population_2021'),
 (0.02045714211710328

# Export Machine Learning Dataset

In [27]:
#export DataFrame to CSV file
df.to_csv('happiness_df.csv', index=False)